Das Notebook ist dafür da die User Stories auszuführen. Bitte testet sie davor in eurem .main (und macht euer main in das gitignore). Hier also nur funktionierende User Stories!

Damit der import richtig funktioniert müsst ihr die init datei des jeweiligen Ordners (model/data_access/business_logic) pflegen.

In [ ]:
import os
os.environ["DB_FILE"] = os.path.abspath("database/hotel_reservation_sample.db")

import model
import data_access
import business_logic

Hotel_manager = business_logic.HotelManager()

1. Als Gast möchte ich die verfügbaren Hotels durchsuchen, damit 
ich dasjenige auswählen kann, welches meinen Wünschen 
entspricht. Wünsche sind:

1.1. Ich möchte alle Hotels in einer Stadt durchsuchen, 
damit ich das Hotel nach meinem bevorzugten Standort 
(Stadt) auswählen kann.

In [ ]:
by_city = Hotel_manager.get_hotels_by_city("Zürich")
print(by_city)

1.2. Ich möchte alle Hotels in einer Stadt nach der
Anzahl der Sterne (z.B. mindestens 4 Sterne) durchsuchen.

In [ ]:
by_city_and_stars = Hotel_manager.get_hotels_by_city_and_stars("Zürich",6)
print(by_city_and_stars)

1.3. Ich möchte alle Hotels in einer Stadt durchsuchen,
die Zimmer haben, die meiner Gästezahl entsprechen (nur 1
Zimmer pro Buchung).

In [ ]:
by_max_guests = Hotel_manager.get_hotels_by_city_and_max_guests("Zürich", 3)
print(by_max_guests)

1.4. Ich möchte alle Hotels in einer Stadt durchsuchen,
die während meines Aufenthaltes ("von" (check_in_date)
und "bis" (check_out_date)) Zimmer zur Verfügung haben,
damit ich nur relevante Ergebnisse sehe.

1.5. Ich möchte Wünsche kombinieren können, z.B. die
verfügbaren Zimmer zusammen mit meiner Gästezahl und der
mindest Anzahl Sterne.

1.6. Ich möchte die folgenden Informationen pro Hotel
sehen: Name, Adresse, Anzahl der Sterne.

In [ ]:
show_user_friendly = Hotel_manager.read_hotel_userfriendly()
for hotels in show_user_friendly:
    print(hotels)

2. Als Gast möchte ich Details zu verschiedenen Zimmertypen
(Single, Double, Suite usw.), die in einem Hotel verfügbar
sind, sehen, einschliesslich der maximalen Anzahl von Gästen
für dieses Zimmer, Beschreibung, Preis und Ausstattung, um eine
fundierte Entscheidung zu treffen.

2.1. Ich möchte die folgenden Informationen pro Zimmer 
sehen: Zimmertyp, max. Anzahl der Gäste, Beschreibung, 
Ausstattung, Preis pro Nacht und Gesamtpreis.

2.2. Ich möchte nur die verfügbaren Zimmer sehen, sofern 
ich meinen Aufenthalt (von – bis) spezifiziert habe

3. Als Admin des Buchungssystems möchte ich die Möglichkeit haben, 
Hotelinformationen zu pflegen, um aktuelle Informationen im 
System zu haben.

3.1. Ich möchte neue Hotels zum System hinzufügen

3.2. Ich möchte Hotels aus dem System entfernen


3.3. Ich möchte die Informationen bestimmter Hotels 
aktualisieren, z. B. den Namen, die Sterne usw.

4. Als Gast möchte ich ein Zimmer in einem bestimmten Hotel 
buchen, um meinen Urlaub zu planen.

5. Als Gast möchte ich nach meinem Aufenthalt eine Rechnung 
erhalten, damit ich einen Zahlungsnachweis habe.
Hint: Fügt einen Eintrag in der «Invoice» Tabelle hinzu.

6. Als Gast möchte ich meine Buchung stornieren, damit ich nicht 
belastet werde, wenn ich das Zimmer nicht mehr benötige.
Hint: Sorgt für die entsprechende Invoice. 

7. Als Gast möchte ich eine dynamische Preisgestaltung auf der 
Grundlage der Nachfrage sehen, damit ich ein Zimmer zum besten 
Preis buchen kann.
Hint: Wendet in der Hochsaison höhere und in der Nebensaison 
niedrigere Tarife an.

8. Als Admin des Buchungssystems möchte ich alle Buchungen aller 
Hotels sehen können, um eine Übersicht zu erhalten.

9. Als Admin möchte ich eine Liste der Zimmer mit ihrer 
Ausstattung sehen, damit ich sie besser bewerben kann.


10.Als Admin möchte ich in der Lage sein, Stammdaten zu verwalten, 
z.B. Zimmertypen, Einrichtungen, und Preise in Echtzeit zu 
aktualisieren, damit das Backend-System aktuelle Informationen 
hat. Hint: Stammdaten sind alle Daten, die nicht von anderen Daten 
abhängen.